In [51]:
import sys
import os
import pandas as pd
import joblib
# Add your project folder path to sys.path
sys.path.append(os.path.abspath("F:/Customer-Retention-Prediction"))

In [42]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV ,StratifiedKFold
from utils.utils import save_model_or_data,load_data
from sklearn.metrics import accuracy_score

lgbm = LGBMClassifier(random_state=42)

In [ ]:
X_train=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\X_train_scaled.csv")
y_train=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\y_train.csv")
X_val=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\X_val_scaled.csv")
y_val=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\y_val.csv")  

##### Feature selection

In [44]:
lgbm.fit(X_train, y_train)

# Get feature importances
importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgbm.feature_importances_
}).sort_values(by='importance', ascending=False)

print(importances)

f:\Customer-Retention-Prediction\.venv\lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
f:\Customer-Retention-Prediction\.venv\lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 338, number of negative: 302
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022330 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1046
[LightGBM] [Info] Number of data points in the train set: 640, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.528125 -> initscore=0.112619
[LightGBM] [Info] Start training from score 0.112619
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [45]:
# Keep top N features
top_features = importances['feature'].iloc[:2]  # top 4
X_train_top = X_train[top_features]
X_val_top = X_val[top_features]

In [46]:
# Step 2: Train new model on reduced features
lgbm_top = LGBMClassifier(random_state=42)
lgbm_top.fit(X_train_top, y_train)

# Step 3: Evaluate
val_acc = lgbm_top.score(X_val_top, y_val)
print("Validation accuracy:", val_acc)

f:\Customer-Retention-Prediction\.venv\lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
f:\Customer-Retention-Prediction\.venv\lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 338, number of negative: 302
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015050 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 428
[LightGBM] [Info] Number of data points in the train set: 640, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.528125 -> initscore=0.112619
[LightGBM] [Info] Start training from score 0.112619
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

In [47]:
lgbm_final = LGBMClassifier(random_state=42)


In [48]:
param_grid = {
    'num_leaves': [31, 50],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [200, 400, 600],
    'max_depth': [-1, 10, 20]
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(lgbm_final, param_grid, cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)

# Pass eval_set and eval_metric via fit_params
fit_params = {
    "eval_set": [(X_val_top, y_val)],
    "eval_metric": "logloss"
}

grid.fit(X_train_top, y_train, **fit_params)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


f:\Customer-Retention-Prediction\.venv\lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
f:\Customer-Retention-Prediction\.venv\lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
f:\Customer-Retention-Prediction\.venv\lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 338, number of negative: 302
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000071 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 428
[LightGBM] [Info] Number of data points in the train set: 640, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.528125 -> initscore=0.112619
[LightGBM] [Info] Start training from score 0.112619
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=LGBMClassifier(random_state=42), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1, 0.2],
                         'max_depth': [-1, 10, 20],
                         'n_estimators': [200, 400, 600],
                         'num_leaves': [31, 50]},
             scoring='accuracy', verbose=1)

In [49]:
print("Best Parameters:", grid.best_params_)
print("Best CV Score:", grid.best_score_)

best_lgbm = grid.best_estimator_

y_val_pred = best_lgbm.predict(X_val_top)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))

Best Parameters: {'learning_rate': 0.05, 'max_depth': -1, 'n_estimators': 200, 'num_leaves': 31}
Best CV Score: 0.5421875
Validation Accuracy: 0.5


In [50]:
y_train_pred = best_lgbm.predict(X_train_top)
print("Training Accuracy:", accuracy_score(y_train, y_train_pred))

Training Accuracy: 0.8953125


In [52]:
# Save model
joblib.dump(best_lgbm, "../models/lgbm_best.pkl")
print("✅ LGBM model saved!")

✅ LGBM model saved!


In [53]:
X_test=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\X_test_scaled.csv")
y_test=pd.read_csv("F:\\Customer-Retention-Prediction\\data\\Processed_data\\y_test.csv")
X_test_top = X_test[top_features]

In [58]:
save_model_or_data(X_test_top,"F:\\Customer-Retention-Prediction\\data\\Processed_data\\X_test_final_lgbm.csv")

✅ DataFrame saved at F:\Customer-Retention-Prediction\data\Processed_data\X_test_final_lgbm.csv


In [54]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
y_pred= best_lgbm.predict(X_test_top)

In [55]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.49      0.50      0.50        94
           1       0.55      0.55      0.55       106

    accuracy                           0.53       200
   macro avg       0.52      0.52      0.52       200
weighted avg       0.53      0.53      0.53       200



In [56]:
print("ROC-AUC:", roc_auc_score(y_test, y_pred))

ROC-AUC: 0.5235849056603774


In [57]:
print(confusion_matrix(y_test, y_pred))

[[47 47]
 [48 58]]


In [ ]:
'''
TN = 47 → 47 customers correctly predicted not churn.

FP = 47 → 47 customers predicted will churn but actually did not.

FN = 48 → 48 customers who did churn but model missed them.

TP = 58 → 58 customers correctly predicted will churn
'''

```
LightGBM seems to be predicting both classes almost equally
```